In [55]:
# Import packages
import requests
import pandas as pd
import glob # File path package 

In [56]:
# Call Etherscan API 
contract_add = '0xa3aee8bce55beea1951ef834b99f3ac60d1abeeb' # VeeFriends Contract Address 
address = "0x6abd9b8ac8797c4227868cef0b1e15d1a1aff48e" # Example

#url = "http://api.etherscan.io/api?module=account&action=tokennfttx&contractaddress=" + contract_add + "&page=1&offset=100&sort=asc&apikey=YourApiKeyToken"
url = "http://api.etherscan.io/api?module=account&action=tokennfttx&contractaddress=" + contract_add + "&address=" + address + "&page=1&offset=100&sort=asc&apikey=YourApiKeyToken"


In [57]:
response = requests.get(url)
address_content = response.json()
address_content

{'status': '1',
 'message': 'OK-Missing/Invalid API Key, rate limit of 1/5sec applied',
 'result': [{'blockNumber': '12421047',
   'timeStamp': '1620842799',
   'hash': '0xca047b9b577c805a0bc919f9d067b9b3665d1ee846c05911a5c9aec50c081f17',
   'nonce': '2',
   'blockHash': '0x21176d166141ce7e379b88c2348064b8907c3d938a6799a09f1c0531ac63afb2',
   'from': '0x0000000000000000000000000000000000000000',
   'contractAddress': '0xa3aee8bce55beea1951ef834b99f3ac60d1abeeb',
   'to': '0x6abd9b8ac8797c4227868cef0b1e15d1a1aff48e',
   'tokenID': '8593',
   'tokenName': 'VeeFriends',
   'tokenSymbol': 'VFT',
   'tokenDecimal': '0',
   'transactionIndex': '309',
   'gas': '280000',
   'gasPrice': '497000000000',
   'gasUsed': '264532',
   'cumulativeGasUsed': '13693819',
   'input': 'deprecated',
   'confirmations': '1268506'},
  {'blockNumber': '12421052',
   'timeStamp': '1620842846',
   'hash': '0xe504f9a19ecd54bdf07d281436b3eee6b59713696534d2b727249b964cd41151',
   'nonce': '3',
   'blockHash': '0x9

<h3><center> Obtain Addresses

In [58]:
path = r'/Users/loomanke/Documents/Github/NFT_Project/DATA' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

# Read in and concat token address csv files 
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [59]:
# Limit data to token address holders 
token_owners = frame["From"].unique()

In [60]:
def divide_chunks(l, n):
    '''
    Input: list of token addresses 
    Output: nested list with 100 addresses in each list
    '''
      
    # looping till length l
    for i in range(0, len(l), n): 
        yield l[i:i + n]

n = 10
divided_token_owners = list(divide_chunks(token_owners, n))
#print(divided_token_owners)

<h3><center> Testing

Goal: Reading in the full list of addresses exceeds the limit of my API call. I subset the data into groups of 100 (nested list). I would like to read in each list object separately so as to not exceed my call limit. 

In [61]:
import random
import time
sleep = 30

apiToken = '6GC1UPX46NE1QPVI9YKI9IPJMFVF6MHKM2' # API token 
eventTransfer = [] # Storing transacation data 
contract_add = '0xa3aee8bce55beea1951ef834b99f3ac60d1abeeb' # VeeFriends Contract Address 

for token_owner in divided_token_owners: 
    for owner in token_owner:
        try: 
            address = str(owner)
            url = "http://api.etherscan.io/api?module=account&action=tokennfttx&contractaddress=" + contract_add + "&address=" + address + "&page=1&offset=100&sort=asc&apikey=" + apiToken
            response = requests.get(url)
            address_content = response.json()
            eventTransfer.append(address_content)
        except requests.exceptions.ConnectionError:
            r.status_code = "Connection refused"
    time.sleep(random.uniform(0, sleep))


In [62]:
pd.set_option('display.max_colwidth', -1) # or 199.
eventTransfer = pd.DataFrame(eventTransfer)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [63]:
# Filter only OK messages for now 
eventTransfer_OK = eventTransfer[eventTransfer['message'] == 'OK']
eventTransfer_OK['message'].value_counts()
eventTransfer_OK = eventTransfer_OK.reset_index()

In [64]:
# Loop through data to transform it to a pd df 
df = pd.DataFrame()
for x in range(len(eventTransfer_OK)): 
    trial = eventTransfer_OK['result'][x]
    df_1 = pd.DataFrame(trial) 
    df = df.append(df_1)

In [65]:
# Reset index 
df = df.reset_index()
# Drop unecessary columns 
df = df.drop('index', 1)
#df = df.drop('level_0', 1)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


In [66]:
df.to_csv (r'/Users/loomanke/Documents/Github/NFT_Project/DATA/partial_transfer_data.csv', index = False, header=True)

In [67]:
df

,blockNumber,timeStamp,hash,nonce,blockHash,from,contractAddress,to,tokenID,tokenName,tokenSymbol,tokenDecimal,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,input,confirmations
0,12417503,1620794381,0x34a1df4e0b54829b5d17a39842dfe6a726e506240e219d981a27ee8fb4af9dd7,10,0x5264dc762b01c09b4020ab9501404b00ac65d001b02668a61a88b6f423ee27a1,0x0000000000000000000000000000000000000000,0xa3aee8bce55beea1951ef834b99f3ac60d1abeeb,0x3cd3bf7db26df10ff229fe8f8650bb9f8d05f103,1565,VeeFriends,VFT,0,246,280000,240000001459,272071,14725096,deprecated,1272050
1,12420222,1620831361,0x7510f1f6b3ac4ac041a49e0258f129aaa44ff9308f58c3600bfc65345d9a8734,21,0xf4e0bc8c1062dea1b6ba8049eebf49487dbd4451a2032e3922afd98d8f9596f5,0x0000000000000000000000000000000000000000,0xa3aee8bce55beea1951ef834b99f3ac60d1abeeb,0x3cd3bf7db26df10ff229fe8f8650bb9f8d05f103,2021,VeeFriends,VFT,0,154,280000,311300000000,274871,9867095,deprecated,1269331
2,12433559,1621009420,0x09974ef814e2e06cf8275736e9592665d1bc2ceb8c702b1ec91c96001a4722e3,22,0xc97baa1a3d3769c76513b0a94732b73aa310ef5e66630e005cdbb6cd7e0d18ef,0x0000000000000000000000000000000000000000,0xa3aee8bce55beea1951ef834b99f3ac60d1abeeb,0x3cd3bf7db26df10ff229fe8f8650bb9f8d05f103,5335,VeeFriends,VFT,0,128,280000,203000000000,274871,8119351,deprecated,1255994
3,12435115,1621029484,0xef5cdb7df7ca8b35031b74659e744c9b52cf3a4f38e6a3846cfe080683d25a27,23,0x519d64b57972f3c6a3f112c581934dd51205ca1b645aaddbdd1e52e1d379458b,0x0000000000000000000000000000000000000000,0xa3aee8bce55beea1951ef834b99f3ac60d1abeeb,0x3cd3bf7db26df10ff229fe8f8650bb9f8d05f103,5981,VeeFriends,VFT,0,168,280000,150000000000,267332,8488897,deprecated,1254438
4,12440542,1621102213,0x39087e18bb2aa2cada83fb551d0285cf1350fc27a51163a959fd69c2e3d944a2,24,0xaa66f61e2d7ed648b71f9760452e17688d9323abb3e1d148e122d030f0525436,0x0000000000000000000000000000000000000000,0xa3aee8bce55beea1951ef834b99f3ac60d1abeeb,0x3cd3bf7db26df10ff229fe8f8650bb9f8d05f103,8078,VeeFriends,VFT,0,143,280000,83000000000,267332,14593733,deprecated,1249011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12159,12486162,1621714058,0x3a6dd0cb6fc9380418743af6dd3f9521ab25c4cbac76af7b34234730de34d718,5,0x4607b2f7a4c6988b9dbcd15df50940ddd1af8c3acc31f45a73846d17ce9c78a3,0x0000000000000000000000000000000000000000,0xa3aee8bce55beea1951ef834b99f3ac60d1abeeb,0x954a53b0ab767be7abfc0fdbca1f5bb32fc1e511,6276,VeeFriends,VFT,0,144,280000,49000000000,267332,13642240,deprecated,1203802
12160,12486197,1621714468,0x13f2fded75aa27d3d293a106228d4d34790795651f03b7a79d93293069b430c5,6,0x94c7c9198ec3d04ca07a1e38397ef34a524d86754895a99037e367ab1dec9fc9,0x0000000000000000000000000000000000000000,0xa3aee8bce55beea1951ef834b99f3ac60d1abeeb,0x954a53b0ab767be7abfc0fdbca1f5bb32fc1e511,1312,VeeFriends,VFT,0,107,280000,48000000000,267332,7269720,deprecated,1203767
12161,12486312,1621715955,0x168ef66548d1be5521f1c083d2ba7b94a7d0067e04b80cf4e057c6adea0787eb,7,0x85a12122827b29e1993181522acc33682aa47547074417b153b9949d2f760c84,0x0000000000000000000000000000000000000000,0xa3aee8bce55beea1951ef834b99f3ac60d1abeeb,0x954a53b0ab767be7abfc0fdbca1f5bb32fc1e511,8932,VeeFriends,VFT,0,153,280000,48000000000,267332,12229313,deprecated,1203652
12162,12499111,1621887017,0xf54d7b151ef193cb4cf1adc1478587eb5ec8f2ded4a61bb989ff2728c4b5ec2f,8,0x2034fa0cdc753b1253e2d8d83979b54026816f34d836cb4bb50d0ff96a2e5a6b,0x0000000000000000000000000000000000000000,0xa3aee8bce55beea1951ef834b99f3ac60d1abeeb,0x954a53b0ab767be7abfc0fdbca1f5bb32fc1e511,5609,VeeFriends,VFT,0,111,280000,110000000000,267332,8738273,deprecated,1190853


<h3><center> Address Information 

In [ ]:
# Current Ether for single address 
/api?module=account&action=balance&address=0xddbd2b932c763ba5b1b7ae3b362eac3e8d40121a&tag=latest&apikey=YourApiKeyToken 

# Current Ether for multiple addresses 
/api?module=account&action=balancehistory&address=0xddbd2b932c763ba5b1b7ae3b362eac3e8d40121a&blockno=8000000&apikey=YourApiKeyToken

# Normal transactions by address 
/api?module=account&action=txlist&address=0xddbd2b932c763ba5b1b7ae3b362eac3e8d40121a&startblock=0&endblock=99999999&sort=asc&apikey=YourApiKeyToken

In [89]:
address = '0x86a457579412f9c0a76b60fe9bf33a9f76d17039'

In [94]:
url = 'http://api.etherscan.io/api?module=account&action=balance&address=' + address + '&tag=latest&apikey=YourApiKeyToken' 


In [95]:
response = requests.get(url)
address_content = response.json()
address_content

{'status': '1',
 'message': 'OK-Missing/Invalid API Key, rate limit of 1/5sec applied',
 'result': '15927892396108346065'}

In [102]:
url = 'https://opensea.io/0x86a457579412f9c0a76b60fe9bf33a9f76d17039'

In [108]:
from bs4 import BeautifulSoup
page = requests.get(url)
page.status_code
soup = BeautifulSoup(page.content, 'html.parser')
soup.find_all('a')

[<a href="https://www.cloudflare.com/5xx-error-landing" id="brand_link" rel="noopener noreferrer" target="_blank">Cloudflare</a>]

In [110]:
url = 'https://etherscan.io/address/0xa2f13e7bdd23f06763b1e567be7b811a48c47236'

In [111]:
page = requests.get(url)
page.status_code

403